# Specify the name of the MongoDB collection

In [1]:
mongo_db_base='UASB03'
mongo_db_collection_name='test2'

# Collect documents from BioRxiv/MedRxiv

In [26]:
%reload_ext autoreload
%autoreload 1
%aimport import_rxiv_program

collect=import_rxiv_program.rxiv_collector(mongo_db_collection_name)

Instance created


### Ask for details about the time window of download, server, etc...

In [27]:
# Détails de la collecte, pour créer l'URL
server = 'biorxiv'# Server biorxiv or medrxiv: 
date_interval = '2021-01-01/2021-01-15' # Date interval (YYYY-MM-DD/YYYY-MM-DD)
counter =0 # Counter to start from (default is 0), allows to restart from a specific point if connection is lost

entry=dict(server=server, date_interval=date_interval, counter=counter)

### Collect archives data

In [29]:
collect.biorxiv_collect(entry_dict=entry)

URL biorxiv:https://api.biorxiv.org/details/biorxiv/2021-01-01/2021-01-15/0
Messages: [{'status': 'ok', 'interval': '2021-01-01:2021-01-15', 'cursor': 0, 'count': 100, 'count_new_papers': '1369', 'total': '1976'}]
Count of new papers: 1369, Total: 1976
Cursor: 0
URL biorxiv:https://api.biorxiv.org/details/biorxiv/2021-01-01/2021-01-15/0
True
Cursor: 100
URL biorxiv:https://api.biorxiv.org/details/biorxiv/2021-01-01/2021-01-15/0
True
Cursor: 200
URL biorxiv:https://api.biorxiv.org/details/biorxiv/2021-01-01/2021-01-15/0
True
Cursor: 300
URL biorxiv:https://api.biorxiv.org/details/biorxiv/2021-01-01/2021-01-15/0
True
Cursor: 400
URL biorxiv:https://api.biorxiv.org/details/biorxiv/2021-01-01/2021-01-15/0
True
Cursor: 500
URL biorxiv:https://api.biorxiv.org/details/biorxiv/2021-01-01/2021-01-15/0
True
Cursor: 600
URL biorxiv:https://api.biorxiv.org/details/biorxiv/2021-01-01/2021-01-15/0


KeyboardInterrupt: 

# Collect citations data from OpenCitations

In [30]:
%reload_ext autoreload
%autoreload 1
%aimport add_citations

cit = add_citations.DataCollector(mongo_db_collection_name)

DataCollector initialized - DOI list is empty


In [31]:
cit.collect_and_upload_CitData()

Number of results: 414
10.1093/bib/bbae217
URL OpenCit:https://opencitations.net/meta/api/v1/metadata/doi:10.1093/bib/bbae217
Error collecting citations for DOI 10.1093/bib/bbae217: list index out of range
URL OpenCit:https://opencitations.net/index/api/v2/citations/doi:10.1093/bib/bbae217?sort=desc(timespan)
Response: []
No data available
10.1186/s12915-021-01125-x
URL OpenCit:https://opencitations.net/meta/api/v1/metadata/doi:10.1186/s12915-021-01125-x
Publication Date: 2021-09-09
URL OpenCit:https://opencitations.net/index/api/v2/citations/doi:10.1186/s12915-021-01125-x?sort=desc(timespan)
Response: [{'author_sc': 'no', 'creation': '2022-11-22', 'oci': '061603553633-06501646056', 'citing': 'omid:br/061603553633 doi:10.3390/cells11233725 openalex:W4309721361', 'cited': 'omid:br/06501646056 doi:10.1186/s12915-021-01125-x openalex:W3198795451 pmid:34503492', 'journal_sc': 'no', 'timespan': 'P1Y2M13D'}, {'author_sc': 'no', 'creation': '2022-07-13', 'oci': '062501982467-06501646056', 'ci

KeyboardInterrupt: 

# Calculate bibliometric variables by MongoDB aggregations

In [32]:
%reload_ext autoreload
%aimport aggregations_mongodb

add = aggregations_mongodb.newFields_mongo(mongo_db_collection_name)

Ready to add fields to test2


### Remove duplicates in the collection and create an index on the doi (keep latest version)

In [33]:
add.create_index_doi() # after this step, not possible to use the bulk insert on the collection (errors if doi already exists)

Ready to add new fields to the collection test2 - Unique index on doi created


### Add new fields : publication status, citations number (year 1, year 2 and total), number of authors, length of the title and abstract

In [34]:
add.add_new_fields()

New fields added to the collection test2


# Create an index of the collection on Elastic Search and use for full-text search

In [35]:
%reload_ext autoreload
%aimport elastic_search

# Connect to MongoDB and the ES index - MongoDB base / MongoDB collection to replicate / ES username / ES password
ES=elastic_search.index_ES(mongo_db_base, 'test2', "elastic", "PeCW-NXhUwTzIGJl2zn+")

Connection to MongoDB collection test2 successfully!
Connected to Elasticsearch successfully!
{'name': '3d21e84e44c6', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'L5WbjgSsQdijVtUF3SeAxw', 'version': {'number': '8.13.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '09df99393193b2c53d92899662a8b8b3c55b45cd', 'build_date': '2024-03-22T03:35:46.757803203Z', 'build_snapshot': False, 'lucene_version': '9.10.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


/Users/gaelleba/anaconda3/envs/sparkEnv/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


## Duplicate MongoDB collection into an ElasticSearch index

In [36]:
# Export into an ES index of the MongoDB collection
ES.export_collection_toES('test')

Index test already exists.
Number of documents retrieved from MongoDB Collection(Database(MongoClient(host=['localhost:27018'], document_class=dict, tz_aware=False, connect=True), 'UASB03'), 'test2'): 90
Number of documents in 'test': 5877


## Full-text search in Elastic Search index

In [37]:
# Query to search for CRISPR technology related articles
query_crispr = {
    "query": {
        "bool": {
            "should": [
                {"match_phrase": {"abstract": "CRISPR"}},
                {"match_phrase": {"abstract": "CRISPR-Cas9"}},
                {"match_phrase": {"abstract": "gene editing"}},
                {"match_phrase": {"abstract": "base editing"}},
                {"match_phrase": {"abstract": "prime editing"}},
                {"match_phrase": {"abstract": "genetic engineering"}}
            ]
        }
    }
}

query_longCovid = {
    "query": {
        "bool": {
            "should": [
                {"match_phrase": {"title": "long COVID"}},
                {"match_phrase": {"title": "post-acute sequelae of SARS-CoV-2"}},
                {"match_phrase": {"title": "post-COVID syndrome"}},
                {"match_phrase": {"title": "persistent COVID symptoms"}},
                {"match_phrase": {"title": "COVID brain fog"}}
            ]
        }
    }
}

# Execute query
results=ES.search_ES_index('uasb03_unique_full', query_longCovid)


{'took': 161, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 184, 'relation': 'eq'}, 'max_score': 33.336346, 'hits': [{'_index': 'uasb03_unique_full', '_id': '10.1101/2021.05.07.21256539', '_score': 33.336346, '_ignored': ['abstract.keyword'], '_source': {'title': 'Immune profile of children with post-acute sequelae of SARS-CoV-2 infection (Long Covid)', 'abstract': 'There is increasing reporting by patients organization and researchers of long covid (or post-acute sequelae of SARS-CoV-2 - PASC), characterized by symptoms such as fatigue, dyspnea, chest pain, cognitive and sleeping disturbances, arthralgia and decline in quality of life. Immune system dysregulation with a hyperinflammatory state, direct viral toxicity, endothelial damage and microvascular injury have been proposed as pathologenic mechanisms. Recently, cohorts of children with PASC have been reported in Italy, Sweden and Russia. However, immunological

In [38]:
ES.results_collection_mongoDB('long_covid_docs', results)

Collection long_covid_docs already exists.
Search results integated successfully to a new MongoDB collection


# Text embeddings for titles and abstracts

In [2]:
%reload_ext autoreload
%aimport bioBert_embeddings

# Connect to MongoDB - MongoDB base / MongoDB collection to embed 
Embed=bioBert_embeddings.BioBertEmbeddings(mongo_db_base, 'long_covid_docs')

Pyspark version 3.5.4
:: loading settings :: url = jar:file:/Users/gaelleba/anaconda3/envs/sparkEnv/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/gaelleba/.ivy2/cache
The jars for the packages stored in: /Users/gaelleba/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-045d937d-f336-4991-ab1b-8d2acbcb9307;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;10.4.0 in central
	found org.mongodb#mongodb-driver-sync;5.1.4 in central
	[5.1.4] org.mongodb#mongodb-driver-sync;[5.1.1,5.1.99)
	found org.mongodb#bson;5.1.4 in central
	found org.mongodb#mongodb-driver-core;5.1.4 in central
	found org.mongodb#bson-record-codec;5.1.4 in central
	found com.johnsnowlabs.nlp#spark-nlp_2.12;5.5.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-s3;1.12.500 in central
	found com.amazonaws#aws-java-sdk-kms;1.12.500 in central
	found com.amazonaws#aws-java-sd

In [3]:
# Specify the model to use - the language 'en' is already specified in the function
Embed.WordEmbedding("biomednlp_biomedbert_base_uncased_abstract_fulltext")


root
 |-- doi: string (nullable = true)
 |-- title: string (nullable = true)
 |-- titleLength: integer (nullable = true)
 |-- abstract: string (nullable = true)
 |-- abstractLength: integer (nullable = true)
 |-- merged_text: string (nullable = true)



Number of documents in the df: 184
Top of the dataframe:


+---------------------------+----------------------------------------------------------------------------------------------------+-----------+----------------------------------------------------------------------------------------------------+--------------+----------------------------------------------------------------------------------------------------+
|                        doi|                                                                                               title|titleLength|                                                                                            abstract|abstractLength|                                                                                         merged_text|
+---------------------------+----------------------------------------------------------------------------------------------------+-----------+----------------------------------------------------------------------------------------------------+--------------+----------------------

25/02/11 17:26:38 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.
25/02/11 17:26:38 WARN S3AbortableInputStream: Not all bytes were read from the S3ObjectInputStream, aborting HTTP connection. This is likely an error and may result in sub-optimal behavior. Request only the bytes you need via a ranged GET or drain the input stream after use.


biomednlp_biomedbert_base_uncased_abstract_fulltext download started this may take some time.
Approximate size to download 389.2 MB
Download done! Loading the resource.
[ — ]Using CPUs
[OK!]
Schema after embedding:
root
 |-- doi: string (nullable = true)
 |-- title: string (nullable = true)
 |-- titleLength: integer (nullable = true)
 |-- abstract: string (nullable = true)
 |-- abstractLength: integer (nullable = true)
 |-- merged_text: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)

Size of the vectors in the features column: 768


+----------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                   title|                                                                                                                                                                                                                                                                                                    features|
+-----------------------------------------------------------------------------------------------------------------------

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=524288Kb used=54448Kb max_used=54463Kb free=469839Kb
 bounds [0x00000001091e8000, 0x000000010c768000, 0x00000001291e8000]
 total_blobs=17076 nmethods=16013 adapters=973
 compilation: disabled (not enough contiguous free space left)


{'_id': ObjectId('678bcc5c66a53e67ed4fd81f'), 'doi': '10.1101/2020.10.01.20201699', 'abstract': 'WITHDRAWAL/DISCLAIMER STATEMENTThe authors have withdrawn this manuscript because of issues that have been raised regarding the process of data collection. Therefore, the authors do not wish this work to be cited as reference for the project. If you have any questions, please contact the corresponding author.', 'abstractLength': 310, 'author_corresponding': 'Esperanza Miyake', 'author_corresponding_institution': 'University of Strathclyde', 'authors': 'Miyake, E.; Martin, S.', 'category': 'health informatics', 'date': '2020-10-04', 'jatsxml': 'https://www.medrxiv.org/content/early/2020/10/04/2020.10.01.20201699.source.xml', 'license': 'cc_by_nc_nd', 'numberAuthors': 2, 'publiStatus': 'NP', 'published': 'NA', 'server': 'medrxiv', 'title': "Long Covid: quantitative and qualitative analyses of online Long Haulers' experiences, emotions and practices in the UK", 'titleLength': 119, 'type': 'PUB

# Export collection

In [4]:
%reload_ext autoreload
%aimport export_collection

output_folder_path='/Users/gaelleba/Library/CloudStorage/Dropbox/CNAM/UASB03/UASB03_DEV'

## for Exploratory Data Analysis (.csv table)

In [5]:

export_collection.export_csv(mongo_db_base, 'long_covid_docs', f'{output_folder_path}/data_export', 'long_covid_collection')


Number of documents retrieved in long_covid_docs: 175
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175 entries, 0 to 174
Data columns (total 12 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   doi                               175 non-null    object 
 1   server                            175 non-null    object 
 2   version                           175 non-null    int64  
 3   category                          175 non-null    object 
 4   publiStatus                       175 non-null    object 
 5   date                              175 non-null    object 
 6   author_corresponding_institution  175 non-null    object 
 7   titleLength                       175 non-null    int64  
 8   abstractLength                    175 non-null    int64  
 9   numberAuthors                     175 non-null    int64  
 10  cit1year                          107 non-null    float64
 11  BioBert          

## for supervised learning

In [6]:
# Initialize the class for the pre-treatment for Machine Learning - initial steps are the same for classification or regression
pretreat=export_collection.pretreatment_spark(mongo_db_base, 'long_covid_docs')


Pyspark version 3.5.4


25/02/11 17:27:11 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


184
root
 |-- server: string (nullable = true)
 |-- category_grouped: string (nullable = true)
 |-- month_upload: integer (nullable = true)
 |-- publiStatus: string (nullable = true)
 |-- titleLength: integer (nullable = true)
 |-- abstractLength: integer (nullable = true)
 |-- numberAuthorsLog: double (nullable = true)
 |-- cit1year: integer (nullable = true)
 |-- BioBert_vec: vector (nullable = true)

+-------+--------------------+------------+-----------+-----------+--------------+------------------+--------+--------------------+
| server|    category_grouped|month_upload|publiStatus|titleLength|abstractLength|  numberAuthorsLog|cit1year|         BioBert_vec|
+-------+--------------------+------------+-----------+-----------+--------------+------------------+--------+--------------------+
|medrxiv|             Bioinfo|          10|         NP|        119|           310|0.6931471805599453|    NULL|[-0.0160322654992...|
|medrxiv|           InfectDis|          10|      year1|        12

### Classification

In [7]:
pretreat.classification(f'{output_folder_path}/binary_models_long_covid', 'df_long_covid_classification')


root
 |-- server: string (nullable = true)
 |-- category_grouped: string (nullable = true)
 |-- month_upload: integer (nullable = true)
 |-- publiStatus: string (nullable = true)
 |-- titleLength: integer (nullable = true)
 |-- abstractLength: integer (nullable = true)
 |-- numberAuthorsLog: double (nullable = true)
 |-- BioBert_vec: vector (nullable = true)
 |-- label: integer (nullable = false)
 |-- server_index: double (nullable = false)
 |-- category_grouped_index: double (nullable = false)
 |-- server_encoded: vector (nullable = true)
 |-- month_upload_encoded: vector (nullable = true)
 |-- category_grouped_encoded: vector (nullable = true)
 |-- subsetFeatures: vector (nullable = true)
 |-- scaled_subsetfeatures: vector (nullable = true)
 |-- features: vector (nullable = true)

+-------+--------------------------+------------+-----------+-----------+--------------+------------------+---------------------------------------------------------------------------------------------------

DataFrame[server: string, category_grouped: string, month_upload: int, publiStatus: string, titleLength: int, abstractLength: int, numberAuthorsLog: double, BioBert_vec: vector, label: int, server_index: double, category_grouped_index: double, server_encoded: vector, month_upload_encoded: vector, category_grouped_encoded: vector, subsetFeatures: vector, scaled_subsetfeatures: vector, features: vector]

### Regression

In [8]:
pretreat.regression(f'{output_folder_path}/binary_models_long_covid', 'df_long_covid_regression')


root
 |-- server: string (nullable = true)
 |-- category_grouped: string (nullable = true)
 |-- month_upload: integer (nullable = true)
 |-- publiStatus: string (nullable = true)
 |-- titleLength: integer (nullable = true)
 |-- abstractLength: integer (nullable = true)
 |-- numberAuthorsLog: double (nullable = true)
 |-- cit1year: integer (nullable = true)
 |-- BioBert_vec: vector (nullable = true)
 |-- server_index: double (nullable = false)
 |-- category_grouped_index: double (nullable = false)
 |-- publiStatus_index: double (nullable = false)
 |-- server_encoded: vector (nullable = true)
 |-- month_upload_encoded: vector (nullable = true)
 |-- category_grouped_encoded: vector (nullable = true)
 |-- publiStatus_encoded: vector (nullable = true)
 |-- subsetFeatures: vector (nullable = true)
 |-- scaled_subsetfeatures: vector (nullable = true)
 |-- features: vector (nullable = true)

+-------+--------------------------+------------+-----------+-----------+--------------+---------------

DataFrame[server: string, category_grouped: string, month_upload: int, publiStatus: string, titleLength: int, abstractLength: int, numberAuthorsLog: double, cit1year: int, BioBert_vec: vector, server_index: double, category_grouped_index: double, publiStatus_index: double, server_encoded: vector, month_upload_encoded: vector, category_grouped_encoded: vector, publiStatus_encoded: vector, subsetFeatures: vector, scaled_subsetfeatures: vector, features: vector]